In [551]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Motivation for You**
* Various Algorithms discussed
* Accuracy score compared
* Cross validation score compared
* Execution time compared 
* All at the end of the exercise

In [552]:
#Plotting library
import matplotlib.pyplot as plt 
import seaborn as sns

**Load Data and Browse through it**

In [553]:
#Read the training set and peek it
df = pd.read_csv('../input/train.csv')
df.head()

In [554]:
# The list of columns
df.columns

In [555]:
# Datatype of columns
df.dtypes

In [556]:
# More on the columns
df.info()

* 891 columns in total.
* Age, Embarked and Cabin column has null data. 
* 'Cabin' has just more than 20%. We may drop it. But later.


In [557]:
df.describe() ## Numerical

In [558]:
df.describe(include=['O']) # Objects

* PassengerId is unique. 
* PassengerId doesnot in anyway related to the Survival rate. So we can drop it. Later  

In [559]:
# Check for duplication 
df.iloc[:, 1:].duplicated().sum() 

In [560]:
# Few initialization
sns.set_style('whitegrid')

> Let us start visualizing the data  
> First the survival rate

In [561]:
sns.countplot(x='Survived',data=df,palette='RdBu_r');

* Some 500+ people survived and 300+ people didnot make it 
* That gives atleast better mix of data and data not biased on one side alone

We will start with where the data is available in full - means all column data available and breakdown 'Survived' based on few factors (like 'Sex', 'Pclass', 'Age' etc). 
This is just to get an idea or feel of the data if at all 'Y' is any way related to 'X'
Note the final idea is to develop a model that explains relationship and understanding and should tell a story and not rigorous math.

Also, if we can identify any relationship manually (without any assistance of algorithm) we can use it to cross verify our results with the final test results as generated by algorithm

In [562]:
sns.countplot(x='Sex',data=df,palette='RdBu_r');


Close to 600 males and just above 300 females. But what about their survival rate?

In [563]:
sns.countplot(x='Survived',hue='Sex',data=df,palette='RdBu_r');

Seems most of the men did not make it. To see the exact nos we use pd.crosstab

In [564]:
pd.crosstab(df.Survived,df.Sex,margins=True)

Better we go with Percentage

In [565]:
pd.crosstab(df.Survived,df.Sex,normalize='index')

* 85% - that is a lot of male who did not make it 
* 68% of female made it
* This seems to give a strong indication to classify further on age which may throw  a little bit of insight into the problem given the heuristics that women and children are given more preference in case of any accidents (for escape) 
Above is an assumption and we have to verify it

Before that lets us see if Survival rate depends on Class travelled using the similar plot

In [566]:
sns.countplot(x='Survived',hue='Pclass',data=df,palette='RdBu_r');

* First class seems to be better on survival
* 2nd class 50/50
* Lot of people travelled in 3rd class didnot make it.


Also with the insight so far had let us combine the above 2 factors to see if we can derive anything better

In [567]:
pd.crosstab([df.Pclass, df.Sex], [df.Survived], margins=True)

* Female travelling in 1st Class survived 98% (approx).
* Female travelling in 2nd Class survived little over 92%. 
* For male travelling in 3rd Class over 86% didnot make it.

**Point 1 - ** Female travelling in Pclass1 survived @ 98%

We may dig more.

**Pclass 1**

In [568]:
df[(df.Sex == 'female') & (df.Pclass == 1) & (df.Survived == 0)]

In [569]:
df[(df.Sex == 'female') & (df.Pclass == 1) & (df.Survived == 1)].head(20)

Cabin seems to give a better choice here to understand differences

In [570]:
df[(df.Cabin == 'C49') | (df.Cabin == 'C22 C26')]

* Women and kids didnot. We may consider them as outliers and discard them. But when it is a fact we leave it in the dataset.
* Ok. New born baby made it on 'C22 C26'. Let us see if this kind of relationship is true for new born babies

In [571]:
df_baby = df[np.isfinite(df['Age'])]

Check for minimum age the babies failed to survive

In [572]:
df_baby.Age[(df_baby.Survived == 0)].min()

In [573]:
df_baby[(df_baby.Age < 1.0) & (df_baby.Survived == 0)]

In [574]:
df_baby[(df_baby.Age < 1.0) & (df_baby.Survived == 1)]

**Point 2 - ** New borns are 'Age<1' always survived whatever be the class. Noble men.

In [575]:
# 'C49'
df[(df.Cabin == 'C49')]

*  Looks like a Senior Madam. 
* She travelled alone. May be news not noticed by her.
* Let us see if it is true for people travelling alone.

In [576]:
df[(df.SibSp == 0) & (df.Parch == 0) & (df.Sex == 'female') ].head()

'C49' not much. Looks like a outlier. We may need to remove it from dataset for training. But when it is a fact we leave it in the dataset.

**Pclass 2**

In [577]:
df[(df.Sex == 'female') & (df.Pclass == 2) & (df.Survived == 0)]

For Class 2 if Embarked @ 'S' they didnot make it and Survived otherwise. Verify.

In [578]:
df[(df.Sex == 'female') & (df.Pclass == 2) & (df.Survived == 1)].head()

Not really!

**Pclass 3**

What about 'Female' for 3rd class. Let us give a try. 50/50 with given data. Anything concrete inference possible we can even remove the 'Sex' column for future analysis as only 'male' will be left.

In [579]:
df[(df.Sex == 'female') & (df.Pclass == 3) & (df.Survived == 0)].head(20)

In [580]:
df[(df.Sex == 'female') & (df.Pclass == 3) & (df.Survived == 1)].head(20)

> No luck.
> They seems more random at this point. May be they are called upon later after filling the first and second class passengers on life saving boatsor caught up at the bottom of the ship before they got the news of sinking. 
> Leave 'Sex' column to stay in the dataset for training.

Nothing more can be said other than this

**Conclusion:** Survival 0f 'female' in 1st & 2nd Class and new borns seems to say the commanders followed the certain protocol. Now there arises the call to check where they did the same with 
* Age and 
* Parch & Sibsp (if any)

In [581]:
grid = sns.FacetGrid(df, col='Survived',size=4, aspect=1.8)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

True.
>  **Point3 :** Senior people >75 yrs old are given preference and they survived

In [582]:
#The cutoff
df[df['Survived']==0].max()

74 yrs. We may take 75 yrs.

In [583]:
# Parch & Sibsp
pd.crosstab([df.Parch, df.SibSp], [df.Survived], margins=True)

In [584]:
# For visual
sns.countplot(x='SibSp',hue = 'Parch', data=df);

Most of the people are 
> 1. Alone (537 nos - 60%) or 
> 2. Travelled with their spouse (123 nos - 14%)  
> 3. Rest - 26 %


Other than that we cannot say anything specific at this point (May be value - '0' can help here establishing correlation with 'Survived', but we keep it simple at this point with no more classification inference on results. We leave it to algorithm) 

And what about fare?

In [585]:
grid = sns.FacetGrid(df, col='Survived',size=4.2, aspect=1.6)
grid.map(plt.hist, 'Fare', alpha=.5, bins=40)
grid.add_legend();

No direct inference can be made here

**Data Cleansing**

In [586]:
df.isnull().sum().plot(kind='bar');

Embarked seems to have a null values

In [587]:
df.Embarked.unique()

In [588]:
df[~((df.Embarked == 'S') | (df.Embarked == 'C') | (df.Embarked == 'Q'))]

Both are female and present in Pclass 1. We can safely check and subsitute Embarked with 'S','Q' or 'C' whatever fits there


In [589]:
df[(df.Sex == 'female') & (df.Pclass == 1) & (df.Survived == 1)].Embarked.value_counts()

We substitute with 'S' or 'C'  as people with same ticket number Embarked at same location (see Cabin samples above and sample further on own). Further Pclass 1 always survived and 'S' or 'C' doesnot matter

In [590]:
df[df['Ticket'] == '113572']

In [591]:
df.loc[61,'Embarked'] = 'C'
df.loc[829,'Embarked'] = 'C'

In [592]:
df.isnull().sum()

Now we have only Age as missing values and we have to make smart decisions here
>Missing Age - 177/887 - 20% approx. We may drop it or insert 'Age' values for training.


We can either drop the 'Age' missing rows (information loss) or we can fill it using some clues and stats from the given data (in which we may bias the data). We have to make a choice here and we go filling the data. Looks like we have a clue to fill up the 'Age'. The title in the name. Title like Mr, Mrs, Master, Miss & others indicate the Age bracket. We can now fill them manually using the Title or Title combined with other columns. But what we can think of is to first split the entire problem into **Predicting the Age first and then Survived or not**

You must have identified now something now. Do I need to repeat the same for test data as well?
> Yes. Wouldn't it be wise if we can combine the 'Age' data of the test dataframe as well to predict all missing age in both Train and Test data at one go. That is what we are going to do now.

In [593]:
#Read the test set and peek it
df1 = pd.read_csv('../input/test.csv')
df1.isnull().sum() 

In [594]:
# One more null apart from Age. Let's fix it
df1[~np.isfinite(df1['Fare'])]

Single man in the 3rd class with 3701 embarked @ 'S'
> Let us now see if there are any fare that is matching up with this criteria or close enough


In [595]:
#df is a large dataset and we search ther
df[(df.Pclass == 3) & (df.Embarked == 'S') & (df.SibSp == 0) & (df.Parch == 0) & (df.Sex == 'male') & (df.Age > 50.0)]

Around $7

In [596]:
df1.loc[152, 'Fare'] = 7.0
df1.loc[152, :]

We finally concat the Training dataset and Testing dataset to predict 'Age' for both (Survived removed). Later once 'Age' is predicted we seperate them once again. 

In [597]:
df2 = pd.concat([df,df1],axis = 0)
df2.shape

 Although Cabin seems to have so lot of missing data we may keep it by seperating the Cabin as 'A','B','C','D','E','F' and 'unknown' for others as they may even influence the survival

In [598]:
def cabin_travelled(cabin):
    if not cabin or pd.isnull(cabin):
        return 'Unknown'
    else:
        return cabin[0]

In [599]:
# Lets create a new column for the Cabins
df2['Cabin'] = df2['Cabin'].map(lambda x: cabin_travelled(x))

In [600]:
df2.Cabin.unique()

In [601]:
# Plot the result
df2.Cabin.value_counts().plot(kind='bar')

Now extract the title

In [602]:
# Functions that returns the title from a name. All the name in the dataset has the format "Surname, Title. Name"
def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'

In [603]:
# Lets create a new column for the titles
df2['Title'] = df2['Name'].map(lambda x: get_title(x))

In [604]:
df2.Title.unique()

In [605]:
df2.Title.value_counts()

In [606]:
# Plot the result
df2.Title.value_counts().plot(kind='bar');

Understanding 'Age' distribution for top categories like 'Mr', 'Mrs', 'Miss', 'Master'

In [607]:
# Mr is widely distributed
df2.Age[df2.Title == 'Mr'].plot(kind = 'hist', bins = 40)
df2[df2.Title == 'Mr'].min()

In [608]:
# Mrs is widely distributed
df2.Age[df2.Title == 'Mrs'].plot(kind = 'hist', bins = 40)
df2[df2.Title == 'Mrs'].min()

In [609]:
# Miss is widely distributed
df2.Age[df2.Title == 'Miss'].plot(kind = 'hist', bins = 40);

In [610]:
# Master refered to kids below 12 yrs old  
df2.Age[df2.Title == 'Master'].plot(kind = 'hist', bins = 40);

We can leave out the 'Mr', 'Mrs', 'Miss', 'Master' for now 
> Check for 'Age' presence for others 

In [611]:
df2[~((df2.Title == 'Mr') | (df2.Title == 'Mrs') | (df2.Title == 'Master') | (df2.Title == 'Miss'))] 

 Only one 'Dr' seems to have missing age and given other 6 'Dr' Age our prediction algorithm should be able to take care of this. But we can drop it too. 

In [612]:
df2.drop(df2[df2.PassengerId == 767].index, inplace = True)

In [613]:
df2[df2.Title == 'Dr']

Categorical values

Now that we have dealt with the missing values our next goal is to look for Categorical values and convert these in to numerical form so that the machine can understand and work upon it with ML algorithms

In [614]:
# Let us look at the dataset once again
df2.head()

In [615]:
df2.columns

Catergorical colums list
* Sex
* Embarked
* Title

Convert into numerical columns (binarized one column for each category) and remove redundancy in colums to avoid the dummy trap by removing one column.

In [616]:
# All can be done in one single step
sex = pd.get_dummies(df2['Sex'],drop_first=True)
# And so with Embarked
embarked = pd.get_dummies(df2['Embarked'],drop_first=True)
# And so with Title
title = pd.get_dummies(df2['Title'],drop_first=True)
# And so with Cabin
cabin = pd.get_dummies(df2['Cabin'], prefix='Cabin')
# And so with Class 
pclass = pd.get_dummies(df2['Pclass'], prefix='Pclass')

Let us now concat these to the original dataframe

In [617]:
df2 = pd.concat([df2,sex,embarked,title,cabin, pclass],axis=1)

Finally it is time to finish off the cleaning of data with one final step
Removing the unnecessary columns
Unnecessary column list
* Passengerid
* Name
* Sex
* Ticket
* Embarked
* Title
* Cabin

In [618]:
df2.drop(['PassengerId', 'Sex','Embarked','Name','Ticket', 'Title', 'Cabin', 'Pclass'],axis=1,inplace=True)

**Predicting the 'Age'**

The idea here is to 
* Treat the 'Survived' column independent - remove it for age prediction
* Generate train data with 'Age' available 
* Generate test data where 'Age' needs prediction
* Note 'Age' our target variable
* Finally after prediction construct the train and test dataset

In [619]:
# Age available
df_train = df2[np.isfinite(df2['Age'])]
# Age not available
df_test = df2[~np.isfinite(df2['Age'])]

In [620]:
# Splitting labels
X_train_age_survived = df_train.Survived
X_train_age = df_train.drop(['Age','Survived'], axis=1)
y_train_age = df_train.Age

X_test_age_survived = df_test.Survived
X_test_age = df_test.drop(['Age','Survived'], axis=1)
y_test_age = df_test.Age

In [621]:
df_test

Apply some regression algorithm ('Age' - Continuous) blindfold we will skip explanation for now 

In [622]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train_age, y_train_age.ravel())
y_test_age = regressor.predict(X_test_age)
y_test_age = pd.Series(y_test_age)
y_test_age.index = X_test_age.index

In [623]:
# Train data with Age, Survived
df_train1 = pd.concat([X_train_age,y_train_age, X_train_age_survived],axis=1)
 # Test data with Age, Survived
df_test1 = pd.concat([X_test_age,y_test_age, X_test_age_survived],axis=1)

In [624]:
df_test1.columns

In [625]:
# Rename '0' to 'Age' column
df_test1.rename(columns={0: 'Age'}, inplace=True)

In [626]:
df_test1.columns

'Age' prediction done for both training and test data. 

Now seperating our Train and Test Dataset once again.

In [627]:
df_train2=pd.concat([df_train1[np.isfinite(df_train1['Survived'])],
                               df_test1[np.isfinite(df_test1['Survived'])]],axis=0) 
df_test2=pd.concat([df_train1[~np.isfinite(df_train1['Survived'])],
                               df_test1[~np.isfinite(df_test1['Survived'])]],axis=0) 
# And sort index
df_train2 = df_train2.sort_index()
df_test2 = df_test2.sort_index()

Contructing our train and test dataset

In [628]:
# Training data
X_train = df_train2.iloc[:,:-1]
y_train = df_train2.iloc[:,-1]

# Testing data
X_test = df_test2.iloc[:,:-1]
y_test = df_test2.iloc[:,-1]

**Model Development**

Now that it is Supervised data learning model we have a list of algorithm which can work upon our dataframe from simple to complex each having its own advantages and disadvantages

What we will do is that we will apply different algorithms and understand their performance and then finally choose a one better to predict the final test dataset

In [629]:
# To store execution time of each algorithm fitting method. We will make a comparison later.
import time

**1. Logistic Regression**

In [630]:
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression()
time_in = time.time()
log_clf.fit(X_train,y_train)
time_out = time.time()

In [631]:
# Execution time in secs
time_clf = (time_out - time_in)

In [632]:
# To make predicion we use it like
y_predict_log_clf = log_clf.predict(X_train)

In [633]:
# The logistic classification works on classiying the '0' & '1' based on probablility of max occurance of target variables
# How do I calculate the probability for each of the target labels 
# The predicted variables are classified on max of all probablility calculated
pred = log_clf.predict_proba(X_train)

In [634]:
# How do we know or model is better at prediction
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(accuracy_score(y_train, log_clf.predict(X_train)))

That is 83%. Means we are accurate 83% when the model is tested against its own training  (Y label) data.

In [635]:
# One single parameter is good. 
# But if we want to know more on how did we arrive at this we have one more called 'classification report'
print(classification_report(y_train,log_clf.predict(X_train)))

Recall 
> When the model predicted '0'. It is correct 89% of time when compared with actual '0's and so with '1' @ 76%

Precision
> The model predicted a class ('0'). 'Precision' is now how much % this is right as a % of predicted '0'.


F1-score
> F1 scores are the harmonic means of precision and recall.


What if I want to go further to understand deeper the misclassification? Use confusion matrix

In [636]:
# Confusion Matrix
print(confusion_matrix(y_train,log_clf.predict(X_train)))
# The result shown in diagonal are where actual and predicted values match and in offdiagonal the misclassification


However good the above model may be with the test set it suffers from Overfitting. Means the model is not a general fit but too good fit for the test data and may fail to fit good for the real data
> How do we evaluate the performance for now the out of seen data

In [637]:
# Cross-validation
# The idea now it to split the train data itself into multiple folds (K-fold) 
# and make themselves act like a train/test and we do this in a roundrobin system
# Then finally we average the accuracy (mean, variance) to understand its performance
from sklearn.model_selection import cross_val_score
print(cross_val_score(log_clf, X_train, y_train, cv=10, scoring='accuracy').mean())
print(cross_val_score(log_clf, X_train, y_train, cv=10, scoring='accuracy').std()) 

In [638]:
# Let us store the cross_val_score for future comparison
cols = ['Classifier','AccuracyScore','cross_val_score_mean','cross_val_score_std','Time_clf(sec)']
df_comparison = pd.DataFrame(columns=cols)

df_comparison.loc[len(df_comparison), :] = ['log_clf',
                  accuracy_score(y_train, log_clf.predict(X_train)),
                  cross_val_score(log_clf, X_train, y_train, cv=10, scoring='accuracy').mean(),
                  cross_val_score(log_clf, X_train, y_train, cv=10, scoring='accuracy').std(),
                  time_clf]
df_comparison.head()

ROC - Receiver Operating Characteristic
> An ROC curve is a commonly used way to visualize the performance of a binary classifier

Basically it plots - Total Positive Rate (True Positive predicted of all Positive in training set) vs False Positive Rate (False Positive of all Negative in training set) when the threshold of classification is changed 
> Better the area under the curve is better the model in predicting the values

In [639]:
# Ref: http://www.prolekare.cz/en/journal-of-czech-physicians-article/roc-analysis-and-the-use-of-cost-benefit-analysis-for-determination-of-the-optimal-cut-point-5403?confirm_rules=1
from sklearn.metrics import roc_curve, auc
#import matplotlib.pyplot as plt
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, log_clf.predict(X_train))
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Interpretation from ROC Curve
* 0.9-1.0 : Excellent
* 0.8-0.9 : Very good
* 0.7-0.8 : Good
* 0.6-0.7 : Sufficient
* 0.5-0.6 : Bad
* <0.5 : Model not useful

With ROC of 0.82 our model is very good in predicting the values
> You can change the model for various threshold but that is a seperate business altogether


However there are other models as well and its variants
* K-Nearest Neighbour
* Support Vector Machine 
* Naive Bayes
* Decision Tree

**2. K-Nearest Neighbour**

Simple algo : You are a 'Roman' while in 'Rome'
The idea here is that you pick 'K' nearest neighbours (based on some parameter like Euclidean distance) and classify the target based on majority category.
> Note you may need to scale the data afront to apply the algo as Measurement units weight one better than other and affect final result


In [640]:
# Fitting K-NN to the Training set
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_knn = sc.fit_transform(X_train)

In [641]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
time_in = time.time()
knn_clf.fit(X_train_knn, y_train)  
time_out = time.time()
time_clf = (time_out - time_in)
y_predict_knn_clf = knn_clf.predict(X_train_knn)

In [642]:
# We define a function for evaluation and call it whenever necessary
def print_score(classifier_name, classifier, X_train, y_train, time_clf):
    print(accuracy_score(y_train, classifier.predict(X_train)))
    print(classification_report(y_train, classifier.predict(X_train)))
    print(confusion_matrix(y_train, classifier.predict(X_train)))
    print(cross_val_score(classifier, X_train, y_train, cv=10, scoring='accuracy').mean())
    print(cross_val_score(classifier, X_train, y_train, cv=10, scoring='accuracy').std()) 
    df_comparison.loc[len(df_comparison), :] = [classifier_name,
                  accuracy_score(y_train, classifier.predict(X_train)),
                  cross_val_score(classifier, X_train, y_train, cv=10, scoring='accuracy').mean(),
                  cross_val_score(classifier, X_train, y_train, cv=10, scoring='accuracy').std(),
                  time_clf]

In [643]:
# Again evaluating the parameters
print_score('knn_clf',knn_clf, X_train_knn, y_train, time_clf)

The scores are less as compared to logmodel and can be improved by iterating K from 1 to 20, 30 etc. Plot the 'K' and the cross_val_score and choose K for which the K is higher

The question is that can we automate this process give the K range
> Yes possible. This is called Parameter tuning. 

In [644]:
# Parameter Tuning
# This is done through GridSearchCV
from sklearn.grid_search import GridSearchCV

# Enter the space to be searched
k_range = list(range(1, 31))
print(k_range)

# Create a parameter grid
param_grid = dict(n_neighbors=k_range)
print(param_grid) 

In [645]:
# Instantiate the grid
knn_grid = GridSearchCV(knn_clf, param_grid, cv=10, scoring='accuracy')

In [646]:
# Fit the grid with data
time_in = time.time()
knn_grid.fit(X_train_knn, y_train)
time_out = time.time()
time_clf = (time_out - time_in)

In [647]:
# Now the grid contains the list of scores for all the grid searched
# By default the grid picks up the best estimator on the space and we can 
# continue with the prediction
print(knn_grid.best_score_)
print(knn_grid.best_params_)
print(knn_grid.best_estimator_)

In [648]:
# We may again evaluate the performance  
print_score('knn_grid', knn_grid, X_train, y_train, time_clf)

**3. Support Vector Machine** 

* In general it finds the boundary layer between the category where tha gap is maximum
* Technically it finds a hyperplane that maximizes the distance between the support vectors

In [649]:
# Scaling of the input data is necessary for the model
X_train_svc = sc.fit_transform(X_train)

In [650]:
from sklearn.svm import SVC
svc_clf = SVC(kernel = 'rbf', random_state = 0)
time_in = time.time()
svc_clf.fit(X_train_svc, y_train)
time_out = time.time()
time_clf = (time_out - time_in)

In [651]:
print_score('svc_clf', svc_clf, X_train_svc, y_train, time_clf)

In [652]:
# One can again choose between various options for parameter tuning like
# linear, polynomial and sigmoid
# SVM are good for higher dimensional problem. Although effective in higher dimension
# avoid using them in cases where no of samples is less than features describing them

**4. Naive Bayes**

In [653]:
# Naive Bayes classifiers are a family of simple probabilistic classifiers based on applying Bayes' theorem with strong (naive) independence assumptions between the features.
# Naive Bayes classifiers are highly scalable
# Further it requires a small number of training data to estimate the parameters necessary for classification
# In simple terms find the probability of occurance of an event with given data
# Model probability of occurance of events and if >0.5 take it as '1' and <0.5 otherwise

In [654]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_naive = sc.fit_transform(X_train)

In [655]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
naive_clf = GaussianNB()
time_in = time.time()
naive_clf.fit(X_train_naive, y_train)
time_out = time.time()
time_clf = (time_out - time_in)

In [656]:
print_score('naive_clf', naive_clf, X_train_svc, y_train, time_clf)

**5. Decision Tree**

In [657]:
# We move down the tree like graph of input parameters and split
# until 0/1 decision (target category) is clearly demarkable at the leaf nodes
# The condition of choosing the tree top input parameters and splitting is based on the theory
# of maximum entropy means '0/1' can occur 50/50% at best on splitting for better prediction.

In [658]:
# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeClassifier
dc_clf = DecisionTreeClassifier(criterion = 'entropy', random_state = 42)
time_in = time.time()
dc_clf.fit(X_train, y_train) 
time_out = time.time()
time_clf = (time_out - time_in)

In [659]:
print_score('dc_clf',dc_clf, X_train, y_train, time_clf)

Much better than anything seen before

In [660]:
# The disadvantage includes overfitting with its train set and also when it meets 
# a new data of test set where variance with test model may be high.
# In order to overcome an ensemble of trees with reduced feature and sample 
# data are used to train the dataset which helps in increasing the model performance

Effectively you have multiple estimators and the final value is determined by the voting principle. Such estimators fall under the ensemble methods.
 

Ensemble methods
* Random Forest
* Adaboost    
* Gradient Boosting Classifier
* XGBoost


**5. Random Forest**

Build many decision tree with reduced feature and subset of samples. The final prediction is averaged across all the subset prediction model


In [661]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
time_in = time.time()
rf_clf.fit(X_train, y_train.ravel())
time_out = time.time()
time_clf = (time_out - time_in)

In [662]:
print_score('rf_clf',rf_clf, X_train, y_train, time_clf)

In [663]:
# We have an added advantage here. The feature importances
pd.Series(rf_clf.feature_importances_, index=X_train.columns).sort_values(ascending=False).plot(kind='bar', figsize=(12,6));

More insights here 
> While we could manually identify that the 'Pclass' and 'Sex' influence more the model 'Fare' and 'Age' seems to influence the survival rate equally as well which was not evident during initial analysis.

Boosting   
> Boosting uses subsets of the original data to produce a series of averagely performing models and then boosts their performance by combining them together using a cost function (majority vote).
> Every new subsets contains the elements that were (likely to be) misclassified by previous models.
     

**6. Adaboost**

These are meta estimator meaning they work on the original dataset for training and when on failed cases they adjust weights and add information to the final prediction.

In [664]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier()
time_in = time.time()
ada_clf.fit(X_train, y_train.ravel())          
time_out = time.time()
time_clf = (time_out - time_in)

In [665]:
print_score('ada_clf', ada_clf, X_train, y_train, time_clf)

Not that great. But they are ensemble methods meaning they can take other classifier like Random forest classifier and develop a model

In [666]:
ada_rf_clf = AdaBoostClassifier(RandomForestClassifier())
time_in = time.time()
ada_rf_clf.fit(X_train, y_train.ravel()) 
time_out = time.time()
time_clf = (time_out - time_in)

In [667]:
print_score('ada_rf_clf',ada_rf_clf, X_train, y_train, time_clf)

Definitely they pushed the limits of the base classifier

**7. Gradient Boosting Classifier**

Gradient boosting is an approach where new models are created that predict the residuals or errors of prior models and then added together to make the final prediction.

In [668]:
from sklearn.ensemble import GradientBoostingClassifier
gbc_clf = GradientBoostingClassifier()
time_in = time.time()
gbc_clf.fit(X_train, y_train.ravel())
time_out = time.time()
time_clf = (time_out - time_in)

In [669]:
print_score('gbc_clf',gbc_clf, X_train, y_train, time_clf)

**8. XGBoost**

XGBoost stands for eXtreme Gradient Boosting.
> It utilizes gradient boosting decision tree algorithm for high speed execution and model performance

In [670]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier()
time_in = time.time()
xgb_clf.fit(X_train, y_train.ravel())
time_out = time.time()
time_clf = (time_out - time_in)

In [671]:
print_score('xgb_clf',xgb_clf, X_train, y_train, time_clf)

**9. Voting Classifier**

It uses the mix of all classifiers to arrive at the final prediction. 
> It is averaged either Hard whereas the voitng of majority wins or soft where the prediction is averaged and final prediction arrived


In [672]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('dc_clf', dc_clf), ('rf_clf', rf_clf), ('ada_clf', ada_clf),
                                     ('ada_rf_clf', ada_rf_clf), ('gbc_clf', gbc_clf), ('xgb_clf', xgb_clf)], voting='soft')

In [673]:
time_in = time.time()
voting_clf = eclf.fit(X_train, y_train.ravel())
time_out = time.time()
time_clf = (time_out - time_in)

In [674]:
print_score('voting_clf',voting_clf, X_train, y_train, time_clf)

We may change the weight of the classifiers for better parameter tuning

**Model performance comparison**

In [675]:
df_comparison

Accuracy score comparison

In [676]:
dims = (10, 8)
fig, ax = plt.subplots(figsize=dims)
sns.barplot(ax=ax, y='AccuracyScore',x='Classifier',data=df_comparison);

Cross Validation Score - mean

In [ ]:
fig, ax = plt.subplots(figsize=dims)
sns.barplot(ax=ax, y='cross_val_score_mean',x='Classifier',data=df_comparison);

Cross Validation Score - variance

In [ ]:
fig, ax = plt.subplots(figsize=dims)
sns.barplot(ax=ax, y='cross_val_score_std',x='Classifier',data=df_comparison);

Execution Time for model fit comparison

In [ ]:
fig, ax = plt.subplots(figsize=dims)
sns.barplot(ax=ax, y='Time_clf(sec)',x='Classifier',data=df_comparison);

Now given the comparison of Accuracy score, Cross validation score and Execution time we should be able to decide which one we can choose. 
* Decision Tree Classifiers and Boosting Algorithms are much better than others. 
* Specifically, Decision Tree and Ada+Random Forest performed too good @ 98.4%. 
* When it comes to cross validation score Gradient Boosting and XGBoost performed slightly better means they should be able to predict slight better the test data (out of seen data)
* On time execution Ada_rf, Voting_clf  & knn grid (choosing from grid space) took more time. But Decision tree did solve the data set at blazing fast delivering overall better result andXGBoost as well. 
* On variance, Boositg algorithm like XGBoost and Gradient Boosting performed well. 
* For larger dataset ada_rf is not recomended while other Decision Tree and Boositng algorithm can be made use of

Given the above facts one may choose between Decision Tree, Ada_rf, Gradient Boosting and XGBoost. 

Did you notice one more thing? 98.8%. That means some 11 points are wrong in the prediction. What it could be due to? 
* May be our assumptions are wrong when we processed the data. 
* We dropped a row with 'Dr' or our 'Fare' introduced for test data is wrong or 
* Remember we dropped the Ticket column. May be that is showing up its face now. May be people with same Ticket no Survived and our model didnot take account of it.  
* May be our algorithm didnot predict the correct 'Age'
* Outliers in the data which we didnot account for. Remember only  'C22 C26'  & 'C49' females didnot survive whereas the rest in the Pclass 2 survived. Such cases may exist within the dataset misleading the algorithm.

Want 100%. It left upto you people who wants to go for the extra mile.

But 100% Accuracy is misleading as it may have overfitting and in which case it may very well fail to predict the out of seen data or any new data that it may be tested against. So try to use cross validation score as it is the best for real world scenarios.

Data cleaning more important than anything else. Work on it.

**Prediction**

In [ ]:
y_predict = voting_clf.predict(X_test) 

As simple as that. 
> Wow I am going to submit. Not yet. Why?

 Remember we collected 3 points by our manual reasoning of train dataset. Just cross verify if it is still true for a majority of the test data and then prepare for release. While Machine Learning algorithms can be perfect it needs manual intervention to verify it that it produces the output we expect and why it did that too.

In [ ]:
# Concatenate our predicted value with test data
df_predict = pd.concat([df1, pd.Series(y_predict)], axis = 1)
df_predict.rename(columns={0: 'Survived'}, inplace=True)

**Point 1 - ** Female travelling in Pclass1 survived @ 98%

In [ ]:
df_predict[(df_predict.Sex == 'female') & (df_predict.Pclass == 1) & (df_predict.Survived == 0)]

In [ ]:
df_predict[(df_predict.Sex == 'female') & (df_predict.Pclass == 1) & (df_predict.Survived == 1)].shape

Means to say we areclose to it once again.

 **Point 2 - ** New borns are 'Age<1' always survived whatever be the class. Noble men.

In [ ]:
df_predict[(df_predict.Age < 1.0) & (df_predict.Survived == 0)]


True. All newborns Survived.

**Point3 - ** Senior people >75 yrs old are given preference and they survived

In [ ]:
df_predict[(df_predict.Age > 75.0) & (df_predict.Survived == 0)]

In [ ]:
df_predict[(df_predict.Age > 75.0) & (df_predict.Survived == 1)]

Confirms our observation True

It very well makes sense to submit our data now.

**Submitting Prediction**

We have to just generate a csv file with 'Survived' status for test data 'PassengerId'. 

In [ ]:
# Submit to Kaggle
Titanic_Kaggle_Submit = pd.DataFrame({
        "PassengerId": df_predict.PassengerId,
        "Survived": df_predict.Survived
    })

Titanic_Kaggle_Submit.PassengerId = Titanic_Kaggle_Submit.PassengerId.astype(int)
Titanic_Kaggle_Submit.Survived = Titanic_Kaggle_Submit.Survived.astype(int)

Titanic_Kaggle_Submit.to_csv("Titanic_Kaggle_Submit_voting_clf.csv", index=False)

In [ ]:
'Finished running all classifier'

**Note** The is my first workbook @ Kaggle and may undergo changes in future for better presentation and observation. Kindly comment if there are any mistakes and corrections to be made. Also various other algorithms like Ensemble of Ensemble methods, Bagging, Voting classifier etc will be included in future.